# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

# DataSet

In [2]:
dataset = datasets.ML20m()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [3]:
# k
ks = [5, 10, 50]

# Accuracy
recall = evaluators.RecallEvaluator(test_set, ks)

# Unpopularity
unpopularity = evaluators.UnpopularityEvaluator(test_set, ks)

# F1-Score (Recall-Unpopularity)
f1_score = evaluators.F1ScoreEvaluator(test_set, ks)

# Coverage
coverage = evaluators.CoverageEvaluator(test_set, ks)

# Hubness
hubness = evaluators.HubnessEvaluator(test_set, ks)

# Sampler

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# distributiuons
dist1 = distributions.Empirical()
dist2 = distributions.Gaussian()
dist3 = distributions.Gamma()

# models
model0 = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
model1 = models.MutualProximityCML(n_user, n_item, dist1, n_dim, n_item_sample, n_user_sample).to(device)
model2 = models.MutualProximityCML(n_user, n_item, dist2, n_dim, n_item_sample, n_user_sample).to(device)
model3 = models.MutualProximityCML(n_user, n_item, dist3, n_dim, n_item_sample, n_user_sample).to(device)

# learning late optimizer
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer0 = trainers.BaseTrainer(model0, optimizer0, criterion, sampler, no_progressbar)
trainer1 = trainers.BaseTrainer(model1, optimizer1, criterion, sampler, no_progressbar)
trainer2 = trainers.BaseTrainer(model2, optimizer2, criterion, sampler, no_progressbar)
trainer3 = trainers.BaseTrainer(model3, optimizer3, criterion, sampler, no_progressbar)

# Recall

In [6]:
# train0
trainer0.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [7]:
# train3
trainer3.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [8]:
# CML
trainer0.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.022550,0.044883,0.221410,0,NaN
0,0.040394,0.136241,0.633027,10,0.453970
0,0.033942,0.132032,0.687985,20,0.398651
0,0.037469,0.146726,0.729746,30,0.366123
0,0.045345,0.169216,0.755495,40,0.342996
0,0.055679,0.192778,0.771479,50,0.320181


In [9]:
# MPCML
trainer3.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.022333,0.044723,0.221810,0,NaN
0,0.009933,0.026437,0.271256,10,0.456218
0,0.004295,0.011029,0.235511,20,0.398094
0,0.004150,0.010899,0.242258,30,0.368253
0,0.005745,0.014142,0.261137,40,0.341678
0,0.007846,0.017991,0.282896,50,0.318371


# Unpopularity

In [10]:
# train0
trainer0.fit(n_batch, n_epoch, unpopularity, valid_per_epoch)

In [11]:
# train3
trainer3.fit(n_batch, n_epoch, unpopularity, valid_per_epoch)

In [12]:
# CML
trainer0.valid_scores

,Unpopularity@5,Unpopularity@10,Unpopularity@50,epoch,losses
0,0.011426,0.016841,0.129914,0,NaN
0,0.009787,0.014348,0.127376,10,0.302819
0,0.008404,0.012374,0.125829,20,0.288428
0,0.007252,0.010798,0.124514,30,0.277582
0,0.006309,0.009546,0.123621,40,0.269619
0,0.005468,0.008551,0.123000,50,0.260907


In [13]:
# MPCML
trainer3.valid_scores

,Unpopularity@5,Unpopularity@10,Unpopularity@50,epoch,losses
0,0.024176,0.048045,0.206618,0,NaN
0,0.023904,0.047542,0.203368,10,0.300831
0,0.023527,0.046818,0.199598,20,0.284859
0,0.022985,0.045811,0.195307,30,0.273380
0,0.021909,0.043910,0.189434,40,0.254377
0,0.020242,0.040998,0.182867,50,0.235325


# F1-score

In [14]:
# train0
trainer0.fit(n_batch, n_epoch, f1_score, valid_per_epoch)

In [15]:
# train3
trainer3.fit(n_batch, n_epoch, f1_score, valid_per_epoch)

In [16]:
# CML
trainer0.valid_scores

,F1-score@5,F1-score@10,F1-score@50,epoch,losses
0,0.003046,0.011915,0.176734,0,NaN
0,0.002719,0.010800,0.175154,10,0.251044
0,0.002292,0.009761,0.172362,20,0.230319
0,0.002202,0.009474,0.170730,30,0.216744
0,0.002136,0.009460,0.170061,40,0.202274
0,0.002081,0.009443,0.169073,50,0.188081


In [17]:
# MPCML
trainer3.valid_scores

,F1-score@5,F1-score@10,F1-score@50,epoch,losses
0,0.000701,0.003440,0.152301,0,NaN
0,0.000732,0.003669,0.148938,10,0.220538
0,0.000801,0.003957,0.146970,20,0.209839
0,0.000922,0.004458,0.145910,30,0.193505
0,0.001082,0.005039,0.146076,40,0.180939
0,0.001199,0.005641,0.147935,50,0.168949


# Coverage

In [18]:
# train0
trainer0.fit(n_batch, n_epoch, coverage, valid_per_epoch)

In [19]:
# train3
trainer3.fit(n_batch, n_epoch, coverage, valid_per_epoch)

In [20]:
# CML
trainer0.valid_scores

,coverage@5,coverage@10,coverage@50,epoch,losses
0,0.181853,0.212500,0.293002,0,NaN
0,0.180840,0.209846,0.290637,10,0.172399
0,0.175965,0.206564,0.287790,20,0.160447
0,0.174180,0.203475,0.285714,30,0.150913
0,0.169546,0.199421,0.283012,40,0.141397
0,0.169208,0.198986,0.280598,50,0.133952


In [21]:
# MPCML
trainer3.valid_scores

,coverage@5,coverage@10,coverage@50,epoch,losses
0,0.197732,0.229537,0.312645,0,NaN
0,0.193726,0.225724,0.309604,10,0.159935
0,0.190734,0.223456,0.305888,20,0.149264
0,0.190251,0.221477,0.306226,30,0.141393
0,0.187693,0.219884,0.304488,40,0.132857
0,0.186245,0.215734,0.303475,50,0.126404


# Hubness 

In [22]:
# train0
trainer0.fit(n_batch, n_epoch, hubness, valid_per_epoch)

In [23]:
# train3
trainer3.fit(n_batch, n_epoch, hubness, valid_per_epoch)

In [24]:
# CML
trainer0.valid_scores

,hubness@5,hubness@10,hubness@50,epoch,losses
0,19.418478,18.817209,10.523655,0,NaN
0,19.168588,18.666126,10.533925,10,0.128500
0,19.215548,18.624022,10.548410,20,0.122754
0,19.236827,18.635435,10.570516,30,0.120914
0,19.316187,18.542071,10.556330,40,0.117164
0,19.202115,18.524127,10.582302,50,0.115143


In [25]:
# MPCML
trainer3.valid_scores

,hubness@5,hubness@10,hubness@50,epoch,losses
0,12.065602,12.555527,9.367167,0,NaN
0,12.686494,13.044404,9.428207,10,0.122185
0,13.129875,13.414740,9.492376,20,0.115691
0,13.488672,13.721875,9.547273,30,0.114222
0,13.944389,14.031944,9.545630,40,0.109680
0,14.208301,14.356333,9.622226,50,0.108083
